In [ ]:
!pip install azureml-sdk

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.authentication import InteractiveLoginAuthentication

In [ ]:
import json

azure_id = open('azure-id.json','r')
my_id = json.load(azure_id)['my_id']

In [ ]:
ws = Workspace.create(name="ModeloSQL",
                      subscription_id= my_id,
                      resource_group = "ModeloSQL",
                      location = "centralindia")

In [ ]:
# Load regression model
reg_model = Model.register(model_path="model.pkl",
                                  model_name="date-regressor",
                                  workspace=ws)

In [ ]:
# Create environment with relevant libraries
venv = Environment("env-datepred")
venv.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn','matplotlib'])

In [ ]:
# Load scoring script
inference_config = InferenceConfig(environment=venv,entry_script="score.py")

# Create a new ACI web service configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

# Deploy the model as a web service
service = Model.deploy(workspace=ws,
                       name='service',
                       models=[reg_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

In [ ]:
service.wait_for_deployment()

In [ ]:
# Generate scoring URI in Azure for model evaluation
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri3.json", "w")
file.write(scoreuri)
file.close()